# COVID-19: A Pandemic Visualised
___

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import coronavirus_data
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import datetime
from IPython.display import Markdown as md

plt.style.use('sample_style')

ModuleNotFoundError: No module named 'coronavirus_data'

In [ ]:
# Update csv files with latest jh data
coronavirus_data.update_data()

# Read data, do some qol stuff
df_cases = pd.read_csv('data/confirmed_cases.csv', skiprows=[1])
df_cases.rename(columns={'Province/State': 'Province', 'Country/Region': 'Country', 'Value': 'Cases'}, inplace=True)
df_deaths = pd.read_csv('data/covid_deaths.csv', skiprows=[1])
df_deaths.rename(columns={'Province/State': 'Province', 'Country/Region': 'Country', 'Value': 'Deaths'}, inplace=True)
df = df_cases.merge(df_deaths)
df['Date'] = pd.to_datetime(df['Date'])

df['New_cases'] = df['Cases'].diff(periods=-1)
df['New_deaths'] = df['Deaths'].diff(periods=-1)

# Covert some of the country names into more usable formats
jh_to_standard = {'US': 'USA', 'United Kingdom': 'UK', 'Central African Republic': 'CAR', 'Saint Vincent and the Grenadines': 'St. Vincent Grenadines', 'Congo (Brazzaville)': 'Congo', 'Congo (Kinshasa)': 'DRC', "Cote d'Ivoire": 'Ivory Coast', 'Holy See': 'Vatican', 'Korea, South': 'South Korea', 'Taiwan*': 'Taiwan', 'United Arab Emirates': 'UAE', 'West Bank and Gaza': 'Palestine'}
df = df.replace({'Country': jh_to_standard})

df_provs = df[-df['Province'].isna()]
for country in ['UK', 'France', 'Denmark', 'Netherlands']: # all have overseas territories that should probably be counted separately
    provinces = df_provs[df_provs['Country'] == country]['Province'].unique()
    for province in provinces:
        df.loc[df['Province'] == province, ['Country']] = province
        df.loc[df['Province'] == province, ['Province']] = np.nan
        
restricted_provinces = ['Diamond Princess', 'Grand Princess', 'Recovered']
df = df[-df['Province'].isin(restricted_provinces)]

In [ ]:
today = datetime.datetime.today().date()
last_data_date = (datetime.datetime.today() - datetime.timedelta(days=1)).strftime('%m-%d-%Y')

# nicer reading date formatting
def suffix(d):
    return 'th' if 11<=d<=13 else {1:'st',2:'nd',3:'rd'}.get(d%10, 'th')

def custom_strftime(format, t):
    return t.strftime(format).replace('{S}', str(t.day) + suffix(t.day))

total_cases = df['Cases'].sum()
most_cases = df['Cases'].max()

def get_country_data(df, country):
    df_country = df.loc[df['Country'] == country] # gets data of just that country
    df_country = df_country.groupby('Date').sum() # gets rid of provinces, grouping by date
    return df_country

## Introduction

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7.5))
colours = plt.rcParams['axes.prop_cycle'].by_key()['color']
df_world = df.groupby('Date').sum()
df_world['Cases'].plot(ax=ax1, linewidth=4)
df_world['Deaths'].plot(ax=ax1, linewidth=4)
df_world['Cases'].plot(ax=ax2, logy=True, linewidth=4)
df_world['Deaths'].plot(ax=ax2, logy=True, linewidth=4)
ax1.axvline('02-18-2020', ls='--', c=colours[2], label='South Korea, Iran, Italy Outbreaks')
ax2.axvline('02-18-2020', ls='--', c=colours[2], label='South Korea, Iran, Italy Outbreaks')
ax1.axvline('03-03-2020', ls='--', c=colours[3], label='Western Europe and USA Outbreaks')
ax2.axvline('03-03-2020', ls='--', c=colours[3], label='Western Europe and USA Outbreaks')
ax1.axvline('03-10-2020', ls='--', c=colours[6], label='South America Outbreaks')
ax2.axvline('03-10-2020', ls='--', c=colours[6], label='South America Outbreaks')
ax1.axvline('03-14-2020', ls='--', c=colours[7], label='Africa Outbreaks')
ax2.axvline('03-14-2020', ls='--', c=colours[7], label='Africa Outbreaks')
ax1.grid(b=True, which='major', axis='both')
ax1.legend()
ax1.set_xlabel(None)
ax1.tick_params(axis='both', which='both', labelsize=12)
ax1.set_title('Coronavirus Timeline')
ax2.set_title('Coronavirus Timeline - Logarithmic Scale')
ax2.grid(b=True, which='major', axis='both')
ax2.legend()
ax2.set_xlabel(None)
ax2.tick_params(axis='both', which='both', labelsize=12)
ax2.yaxis.set_major_formatter(mticker.ScalarFormatter())

In [ ]:
today_formatted = custom_strftime('{S} %B', today)
md(f'To date **({today_formatted}),** the coronavirus has infected over **1,200,000 people** across the world, caused more than **65,000 deaths** and is causing immense **social and economic disruption** not seen in the Western world since the Second World War. Entire societies are staying in their homes. Governments are spending billions in attempts to save the livelihoods of their citizens, but be in no doubt that there will be enormous repercussions from this once this is all over. However, what is perhaps most worrying is that, for many countries all around the world, this appears to be just the start of their epidemic which will change the lives of their people for weeks and probably months to come.\n\nTo understand how such a disease can grow to cause the upheaval of society in such a rapid and devastating manner, we should go back to the start. I will make use of data visualizations to help explain the story, draw insights and shed light on important points in this pandemic.')

## China

### An Overall Look

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7.5))
for country in ['China']:
    df_country = get_country_data(df, country)
    df_country['Cases'].plot(ax=ax1, label=f'{country} Cases', linewidth=4)
    ax1.set_title('Confirmed Cases')
    ax1.grid(b=True, which='major', axis='both')
    ax1.set_xlabel(None)
    ax1.tick_params(axis='both', which='both', labelsize=12)
    df_country['Cases'].plot(ax=ax2, logy=True, label=f'{country} Cases', linewidth=4)
    ax2.set_title('Confirmed Cases - Logarithmic Scale')
    ax2.grid(b=True, which='both', axis='y')
    ax2.grid(b=True, which='major', axis='x')
    ax2.set_xlabel(None)
    ax2.yaxis.set_major_formatter(mticker.ScalarFormatter())
    ax2.tick_params(axis='both', which='both', labelsize=12)

We are probably all now very familiar with the distinctive shape of these figures, they've been everywhere. The graph on the left is the total number of confirmed cases in China. The one on the right is this same data plotted with a logarithic y-axis, which is often more intuitive for understanding exponential growth scenarios, like in an epidemic.

**Observations:**
* There is some data missing from the start of the outbreak. The first cases of Covid-19 emerged in China in mid-to-late December.
* The sudden spike in confirmed cases in mid-February (increasing by around 15,000 cases in one day) was because the Chinese authorities changed the way they define a 'confirmed case' - it didn't represent an actual increase in the number of infected people. This hampers the visualization of the Chinese outbreak a little, but in general, the number of infections was flattening out by then so this really shouldn't be a significant event, even if the graph suggests that it might be.
* The spread of the virus appears to have only been exponential up until around 29th January. On the logarithmic y-axis plot, the curve begins to tail off from around then, then almost totally flattening from the start of March.

These graphs give an overall view of the spread of coronavirus in China but there is more to China's story than these graph suggests. To dig a little bit deeper, we can take a look at some of the different provinces in China.

### Chinese Provinces

In [ ]:
def get_all_country_data(df, country):
    df_country = df.loc[df['Country'] == country] # gets data of just that country
    return df_country

df_china = get_all_country_data(df, 'China')
df_china_today = df_china[df_china['Date'] == last_data_date].sort_values(by='Cases', ascending=False)
top_10_provinces = df_china_today.head(10)['Province'].to_list()

def get_province_data(df, province):
    df_province = df.loc[df['Province'] == province]
    return df_province

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7.5))

for province in top_10_provinces:
    province_data = get_province_data(df, province)
    province_data.set_index('Date', inplace=True)
    province_data['Cases'].plot(ax=ax2, linewidth=3)
ax2.set_title('Top 10 Provinces')
ax2.grid(b=True, which='major', axis='both')
ax2.set_xlabel(None)
ax2.tick_params(axis='both', which='both', labelsize=13)

df_china_today = df_china[df_china['Date'] == last_data_date].sort_values(by='Cases', ascending=False)
df_china_today.set_index('Province', inplace=True)
df_china_today.head(15)

ax1.set_title(f'Top 10 Provinces - {today_formatted}')
df_china_today.head(10)['Cases'].plot.bar(ax=ax1)
ax1.grid(b=True, which='major', axis='y')
ax1.set_xlabel(None)
ax1.tick_params(axis='both', which='both', labelsize=13)

It appears that Hubei province, home to the original epicenter of the virus, was responsible for the overwhelming majority of cases in China (83%). This statistic might appear obvious - the epicenter province should have the most cases, right? - but to contain 83% of all of China's cases in one province shows that an immense containment effort must have taken place.

China received a lot of criticism in the early stages of the epidemic for suppressing information regarding the spread of the virus and for acting too slowly. However, in the days and weeks since, China introduced strong measures to contain the spread of the virus to Hubei. The graph above shows the clear success of this effort and further efforts to track, isolate and contain cases in other Chinese provinces, stopping the exponential rise in cases seen in so many countries around the world.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7.5))
provinces = top_10_provinces[1:]
provinces.extend(['Beijing', 'Shanghai'])
for province in provinces:
    province_data = get_province_data(df, province)
    province_data.set_index('Date', inplace=True)
    province_data['Cases'].plot(ax=ax, linewidth=2, label=province)
ax.axvline('03-16-2020', ls='--', c='grey', label='Second Wave of Coronavirus?')
ax.set_title('Confirmed Cases - Top 10 Chinese Provinces Without Hubei')
ax.grid(b=True, which='major', axis='both')
ax.set_xlabel(None)
ax.legend()
ax.tick_params(axis='both', which='both', labelsize=13)

Quickly before we move on from China, I want to draw attention to the other provinces most affected by coronavirus. Looking at the graph above we can see that **only six Chinese provinces reached 1,000 cases**, including Hubei. As of 31st March, **there are now 45 countries around the world with over 1,000 cases** - and this will continue to increase in the coming days - surely there must be something remarkable about China's response.

Interestingly, it appears there may be a 'second-wave' of new coronavirus cases appearing in China, just as social distancing measures are starting to be relaxed. This resurgence is primarily a consequence of imported cases but it is definitely one to watch out for as time goes on and could be an example for other countries to learn from.

Herd immunity - where enough people in a community (around 60%) have developed immunity such that transmission chains are broken and a virus cannot take hold - almost certainly has not occured, even in Hubei, so another outbreak with community transmission is definitely possible in China if social distancing measures are relaxed too quickly.

## Early Outbreaks in Southeast Asia
**Singapore**, **Taiwan**, **Thailand**, **Vietnam** and **Malaysia** all reported their first cases a few weeks into the China outbreak. However, they were all able to track their cases, isolate them and stop a serious outbreak from occuring. Singapore and Taiwan, in particular, had experienced cases of Severe Acute Respiratory Syndrome (SARS-CoV) back in 2003 and had [33 and 37 deaths](https://www.who.int/csr/sars/country/table2004_04_21/en/) respectively. SARS was a particularly lethal virus with an overall [case fatality rate of 9.6%](https://www.who.int/csr/sars/country/table2004_04_21/en/) and it appears that these countries have learned from that dark experience and acted swiftly to trace and isolate cases.

In [ ]:
fig, (ax3, ax4) = plt.subplots(1, 2, figsize=(20, 7.5))
for country in ['Taiwan', 'Thailand', 'Vietnam', 'Malaysia', 'Singapore']:
    df_country = get_country_data(df, country)
    df_country['Cases'].plot(ax=ax3, label=country)
    df_country['Cases'].plot(ax=ax4, logy=True, label=country)
ax3.legend()
ax3.set_title('Confirmed Cases')    
ax3.set_ylim(1, 40) 
ax3.set_xlim('01-22-2020', '02-16-2020') 
ax3.set_xlabel(None)
ax4.legend()
ax4.set_title('Confirmed Cases - Logarithmic Scale')    
ax4.set_ylim(1, 40) 
ax4.set_xlim('01-22-2020', '02-16-2020') 
ax4.set_xlabel(None)
ax3.grid(b=True, which='major', axis='both')
ax4.grid(b=True, which='major', axis='both')
ax4.grid(b=True, which='minor', axis='y')
ax4.yaxis.set_major_formatter(mticker.ScalarFormatter())
ax3.tick_params(axis='both', which='both', labelsize=12)
ax4.tick_params(axis='both', which='both', labelsize=12)

**Observations:**
* Early efforts from Taiwan, Malaysia, Thailand and Vietnam meant that their numbers of cases all looked like they had tailed off and had they had succeeded in containing their outbreaks for now, all managing to keep total cases under 40.
* Singapore, while not stopping new cases completely, certainly managed to prevent the spread from becoming exponential.

## The first footholds outside China

### South Korea, Iran and Italy
Two months since the virus was first made known, somehow the spread of the virus had been mostly contained to China. However, there was an underlying inevitability that sooner or later it would escape, particularly after the revelation of the virus' [5 day incubation period](https://www.sciencedaily.com/releases/2020/03/200317175438.htm), in which infected individuals do not display symptoms but can still infect others.

On 18th February, in Daegu, South Korea, a member of a religious sect was diagnosed with coronavirus, the country's 31st case. In the few days after, tens and then hundreds of new cases from the same religious gatherings were confirmed. By 28th February, there were 2337 confirmed cases.

Meanwhile in Italy, a cluster of cases began to emerge in the Lombardy region, Northern Italy. Similarly, cases started to trickle and then flood in in Iran.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7.5))
for country in ['Italy', 'Iran', 'South Korea']:
    df_country = get_country_data(df, country)
    df_country['Cases'].plot(label=country, ax=ax1)
    df_country['Deaths'].plot(label=country, ax=ax2)
ax1.set_ylim(0, 20000)
ax1.set_xlim('02-17-2020', '03-20-2020')
ax1.set_title('Confirmed Cases')
ax1.grid(b=True, which='major', axis='both')
ax1.legend()
ax2.set_xlim('02-17-2020', '03-16-2020')
ax2.set_title('Deaths')
ax2.grid(b=True, which='major', axis='both')
ax2.legend()
ax2.set_ylim(0, 3000)
ax1.set_xlabel(None)
ax2.set_xlabel(None)
ax1.tick_params(axis='both', which='both', labelsize=12)
ax2.tick_params(axis='both', which='both', labelsize=12)

**Observations:**
* South Korea used an app to track new infections and help citizens avoid those who are known to have coronavirus
* Italy has suffered most of these three according to the figures.

### Western Europe and USA
In the week or two after Italy's outbreak began to take off, cases started to emerge in France, Spain, Germany and USA. Around a week later, UK, Switzerland and other countries in Europe saw similar growth in confirmed cases. On 11th March, the [World Health Organization (WHO) declared COVID-19 a pandemic](https://www.who.int/dg/speeches/detail/who-director-general-s-opening-remarks-at-the-media-briefing-on-covid-19---11-march-2020).

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 15))
for country in ['Italy', 'France', 'Spain', 'USA', 'Germany']:
    df_country = get_country_data(df, country)
    df_country['Cases'].plot(label=country, ax=ax1)
    df_country['Deaths'].plot(label=country, ax=ax2)
    df_country['Cases'].plot(label=country, logy=True, ax=ax3)
    df_country['Deaths'].plot(label=country, logy=True, ax=ax4)
#ax1.set_ylim(0, 40000)
ax1.set_xlim('02-17-2020', last_data_date)
ax1.set_title('Confirmed Cases')
ax1.grid(b=True, which='major', axis='both')
ax1.legend()
ax2.set_xlim('02-17-2020', last_data_date)
ax2.set_title('Deaths')
ax2.grid(b=True, which='major', axis='both')
ax2.legend()
ax1.set_xlabel(None)
ax2.set_xlabel(None)
#ax2.set_ylim(0, 6000)
ax1.tick_params(axis='both', which='both', labelsize=12)
ax2.tick_params(axis='both', which='both', labelsize=12)
ax3.set_xlim('02-17-2020', last_data_date)
ax3.set_title('Confirmed Cases - Logarithmic Scale')
ax3.grid(b=True, which='major', axis='both')
ax3.legend()
ax4.set_xlim('02-17-2020', last_data_date)
ax4.set_title('Deaths - Logarithmic Scale')
ax4.grid(b=True, which='major', axis='both')
ax4.legend()
ax3.set_xlabel(None)
ax4.set_xlabel(None)
ax3.tick_params(axis='both', which='both', labelsize=12)
ax4.tick_params(axis='both', which='both', labelsize=12)
ax3.yaxis.set_major_formatter(mticker.ScalarFormatter())
ax4.yaxis.set_major_formatter(mticker.ScalarFormatter())

**Observations:**
* Note that Germany has far fewer deaths than France, Spain or USA, despite having similar figures for confirmed cases. Is this because the Germans somehow have stronger immune systems than the French or Spanish? Of course not! Germany has tested many more people per capita for coronavirus than Spain, France and USA. The effect of this is that milder cases are being picked up in Germany where they would go unnoticed elsewhere. Germany has far fewer deaths which actually suggests that **Germany's epidemic is lagging behind** the epidemics in Spain, France, USA and even UK - which has far fewer confirmed cases -  despite what the data might suggest at a glance.
* The difference in testing figures from country to country is important - **comparing countries on confirmed cases alone can be misleading**.
* **The number of deaths in a country is a more stable and reliable statistic** in epidemiology, particularly when comparing countries where healthcare is of a similar standard (This breaks down when comparing countries with varying healthcare standards, where the number of deaths per case would be expected to vary).

## Lock-down
Across the world, various goverments have imposed periods of lock-down to try to break chains of infection and reduce the number of new cases. It is too soon to appreciate the impact of these measures in some countries but already we can see Italy's curve beginning to tail off as a result.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7.5))
for country in ['Italy']:
    df_country = get_country_data(df, country)
    df_country['Cases'].plot(ax=ax1)
    df_country['Deaths'].plot(ax=ax1)
    df_country['Cases'].plot(ax=ax2, logy=True)
    df_country['Deaths'].plot(ax=ax2, logy=True)    
ax1.axvline('03-09-2020', ls='--', c='grey', label='Lock-down Imposed')
ax2.axvline('03-09-2020', ls='--', c='grey', label='Lock-down Imposed')
ax1.set_xlim('02-17-2020', last_data_date)
ax1.set_title(f'{country}')
ax1.grid(b=True, which='major', axis='both')
ax1.legend()
ax1.set_xlabel(None)
ax1.tick_params(axis='both', which='both', labelsize=12)
ax2.set_xlim('02-17-2020', last_data_date)
ax2.set_title(f'{country} - Logarithmic Scale')
ax2.grid(b=True, which='major', axis='both')
ax2.legend()
ax2.set_xlabel(None)
ax2.tick_params(axis='both', which='both', labelsize=12)
ax2.yaxis.set_major_formatter(mticker.ScalarFormatter())

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7.5))
for country in ['Spain']:
    df_country = get_country_data(df, country)
    df_country['Cases'].plot(ax=ax1)
    df_country['Deaths'].plot(ax=ax1)
    df_country['Cases'].plot(ax=ax2, logy=True)
    df_country['Deaths'].plot(ax=ax2, logy=True)
ax1.axvline('03-15-2020', ls='--', c='grey', label='Lock-down Imposed')
ax2.axvline('03-15-2020', ls='--', c='grey', label='Lock-down Imposed')
ax1.set_xlim('03-04-2020', last_data_date)
ax1.set_title(f'{country}')
ax1.grid(b=True, which='major', axis='both')
ax1.legend()
ax1.set_xlabel(None)
ax1.tick_params(axis='both', which='both', labelsize=12)
ax2.set_xlim('03-04-2020', last_data_date)
ax2.set_title(f'{country} - Logarithmic Scale')
ax2.grid(b=True, which='major', axis='both')
ax2.legend()
ax2.set_xlabel(None)
ax2.tick_params(axis='both', which='both', labelsize=12)
ax2.yaxis.set_major_formatter(mticker.ScalarFormatter())

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7.5))
for country in ['USA']:
    df_country = get_country_data(df, country)
    df_country['Cases'].plot(ax=ax1)
    df_country['Deaths'].plot(ax=ax1)
    df_country['Cases'].plot(ax=ax2, logy=True)
    df_country['Deaths'].plot(ax=ax2, logy=True)
ax1.axvline('03-22-2020', ls='--', c='grey', label=' New York Lock-down')
ax2.axvline('03-22-2020', ls='--', c='grey', label='New York Lock-down')
ax1.set_xlim('03-07-2020', last_data_date)
ax1.set_title(f'{country}')
ax1.grid(b=True, which='major', axis='both')
ax1.legend()
ax1.set_xlabel(None)
ax1.tick_params(axis='both', which='both', labelsize=12)
ax2.set_xlim('03-07-2020', last_data_date)
ax2.set_title(f'{country} - Logarithmic Scale')
ax2.grid(b=True, which='major', axis='both')
ax2.legend()
ax2.set_xlabel(None)
ax2.tick_params(axis='both', which='both', labelsize=12)
ax2.yaxis.set_major_formatter(mticker.ScalarFormatter())

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7.5))
for country in ['France']:
    df_country = get_country_data(df, country)
    df_country['Cases'].plot(ax=ax1)
    df_country['Deaths'].plot(ax=ax1)
    df_country['Cases'].plot(ax=ax2, logy=True)
    df_country['Deaths'].plot(ax=ax2, logy=True)
ax1.axvline('03-17-2020', ls='--', c='grey', label='Lock-down Imposed')
ax2.axvline('03-17-2020', ls='--', c='grey', label='Lock-down Imposed')
ax1.set_xlim('02-27-2020', last_data_date)
ax1.set_title(f'{country}')
ax1.grid(b=True, which='major', axis='both')
ax1.legend()
ax1.set_xlabel(None)
ax1.tick_params(axis='both', which='both', labelsize=12)
ax2.set_xlim('02-27-2020', last_data_date)
ax2.set_title(f'{country} - Logarithmic Scale')
ax2.grid(b=True, which='major', axis='both')
ax2.legend()
ax2.set_xlabel(None)
ax2.tick_params(axis='both', which='both', labelsize=12)
ax2.yaxis.set_major_formatter(mticker.ScalarFormatter())

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7.5))
for country in ['UK']:
    df_country = get_country_data(df, country)
    df_country['Cases'].plot(ax=ax1)
    df_country['Deaths'].plot(ax=ax1)
    df_country['Cases'].plot(ax=ax2, logy=True)
    df_country['Deaths'].plot(ax=ax2, logy=True)
ax1.axvline('03-16-2020', ls='--', c='grey', label='First Social Distancing Order')
ax2.axvline('03-16-2020', ls='--', c='grey', label='First Social Distancing Order')
ax1.axvline('03-23-2020', ls='-.', c='grey', label='Lock-down Imposed')
ax2.axvline('03-23-2020', ls='-.', c='grey', label='Lock-down Imposed')
ax1.set_xlim('03-01-2020', last_data_date)
ax1.set_title('United Kingdom')
ax1.grid(b=True, which='major', axis='both')
ax1.legend()
ax1.set_xlabel(None)
ax1.tick_params(axis='both', which='both', labelsize=12)
ax2.set_xlim('03-01-2020', last_data_date)
ax2.set_title('United Kingdom - Logarithmic Scale')
ax2.grid(b=True, which='major', axis='both')
ax2.legend()
ax2.set_xlabel(None)
ax2.tick_params(axis='both', which='both', labelsize=12)
ax2.yaxis.set_major_formatter(mticker.ScalarFormatter())

## Coronavirus spreads all around the world
### Indonesia
Indonesia was an perplexing case study for the duration of February and the start of March; even though it had many flights to and from Wuhan, the epicenter of the virus, as well as many other flights to and from China, its first cases didn't appear for a long time. Many people speculated that this was because of the lack of testing performed by the government but the weeks went on without any sign of coronavirus. In a country with 264 million people and with varying standards of health care, this was a relief.

However, on 2nd March, Indonesia reported their first two cases of coronavirus. Before long, the country which had somehow evaded coronavirus now had its own epidemic. Indonesia currently has one of the highest case fatality rates at 8.9% - whether this is due to undertesting or partly lower healthcare capacity, this is extremely worrying either way.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7.5))
for country in ['Indonesia']:
    df_country = get_country_data(df, country)
    df_country['Cases'].plot(ax=ax1)
    df_country['Deaths'].plot(ax=ax1)
    df_country['Cases'].plot(ax=ax2, logy=True)
    df_country['Deaths'].plot(ax=ax2, logy=True)
#ax1.axvline('03-22-2020', ls='--', c='grey', label=' New York Lock-down')
#ax2.axvline('03-22-2020', ls='--', c='grey', label='New York Lock-down')
ax1.set_xlim('03-01-2020', last_data_date)
ax1.set_title(f'{country}')
ax1.grid(b=True, which='major', axis='both')
ax1.legend()
ax1.set_xlabel(None)
ax1.tick_params(axis='both', which='both', labelsize=12)
ax2.set_xlim('03-01-2020', last_data_date)
ax2.set_title(f'{country} - Logarithmic Scale')
ax2.grid(b=True, which='major', axis='both')
ax2.legend()
ax2.set_xlabel(None)
ax2.tick_params(axis='both', which='both', labelsize=12)
ax2.yaxis.set_major_formatter(mticker.ScalarFormatter())

### Brazil
Brazil is another developing country with a large population at 209 million people. Despite Brazilian President Jair Bolsonaro's [repeated denials that coronavirus is a threat](https://www.bbc.co.uk/news/world-latin-america-52080830), Brazil's confirmed cases have now reached over 4,600 and 165 deaths. The numbers do appear to be moving away from the initial sharp increase in deaths, although the number of cases and deaths will surely continue to increase for several weeks, as in other countries around the world.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7.5))
for country in ['Brazil']:
    df_country = get_country_data(df, country)
    df_country['Cases'].plot(ax=ax1)
    df_country['Deaths'].plot(ax=ax1)
    df_country['Cases'].plot(ax=ax2, logy=True)
    df_country['Deaths'].plot(ax=ax2, logy=True)
#ax1.axvline('03-22-2020', ls='--', c='grey', label=' New York Lock-down')
#ax2.axvline('03-22-2020', ls='--', c='grey', label='New York Lock-down')
ax1.set_xlim('03-05-2020', last_data_date)
ax1.set_title(f'{country}')
ax1.grid(b=True, which='major', axis='both')
ax1.legend()
ax1.set_xlabel(None)
ax1.tick_params(axis='both', which='both', labelsize=12)
ax2.set_xlim('03-05-2020', last_data_date)
ax2.set_title(f'{country} - Logarithmic Scale')
ax2.grid(b=True, which='major', axis='both')
ax2.legend()
ax2.set_xlabel(None)
ax2.tick_params(axis='both', which='both', labelsize=12)
ax2.yaxis.set_major_formatter(mticker.ScalarFormatter())

### Singapore and Taiwan
Despite being among the first countries in the world to acquire cases of coronavirus, Singapore and Taiwan were initially able to control and reduce the spread of the virus in their countries. However, as the epidemic truly became global and the scale of the pandemic increased, cases were finally able to spread here, just as they have all over the world.

**Observations:**
* Interestingly, Singapore was able to avoid its first coronavirus deaths until 21st March despite having over 400 confirmed cases.
* Taiwan's growth up until mid-March is abnormally linear compared to almost every other country; this is suggestive of Taiwan's very fast response to new cases, rigorously tracing the contacts of infected people to ensure that an outbreak could not occur.
* There are already strong signs in the data that both countries are having success in controlling their most recent outbreaks.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7.5))
for country in ['Singapore']:
    df_country = get_country_data(df, country)
    df_country['Cases'].plot(ax=ax1)
    df_country['Deaths'].plot(ax=ax1)
    df_country['Cases'].plot(ax=ax2, logy=True)
    df_country['Deaths'].plot(ax=ax2, logy=True)
#ax1.axvline('03-22-2020', ls='--', c='grey', label=' New York Lock-down')
#ax2.axvline('03-22-2020', ls='--', c='grey', label='New York Lock-down')
ax1.set_title(f'{country}')
ax1.grid(b=True, which='major', axis='both')
ax1.legend()
ax1.set_xlabel(None)
ax1.tick_params(axis='both', which='both', labelsize=12)
ax2.set_title(f'{country} - Logarithmic Scale')
ax2.grid(b=True, which='major', axis='both')
ax2.legend()
ax2.set_xlabel(None)
ax2.tick_params(axis='both', which='both', labelsize=12)
ax2.yaxis.set_major_formatter(mticker.ScalarFormatter())

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7.5))
for country in ['Taiwan']:
    df_country = get_country_data(df, country)
    df_country['Cases'].plot(ax=ax1)
    df_country['Deaths'].plot(ax=ax1)
    df_country['Cases'].plot(ax=ax2, logy=True)
    df_country['Deaths'].plot(ax=ax2, logy=True)
ax1.set_title(f'{country}')
ax1.grid(b=True, which='major', axis='both')
ax1.legend()
ax1.set_xlabel(None)
ax1.tick_params(axis='both', which='both', labelsize=12)
ax2.set_title(f'{country} - Logarithmic Scale')
ax2.grid(b=True, which='major', axis='both')
ax2.legend()
ax2.set_xlabel(None)
ax2.tick_params(axis='both', which='both', labelsize=12)
ax2.yaxis.set_major_formatter(mticker.ScalarFormatter())

### Africa
It was initially hoped that Africa might be able to avoid cases as global transport links shut down. However, looking at the current figures, there are now cases in most countries across the African continent and according to the latest WHO situation report, there is [community transmission in 22 African countries](https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200330-sitrep-70-covid-19.pdf?sfvrsn=7e0fe3f8_2). With a continent population of 1.2 billion this is extremely worrying. With every major country on the planet now facing its own coronavirus crisis, Sub-Saharan African nations will likely be left to fend for themselves.

In [ ]:
africa = ['Burkina Faso', 'Senegal', 'Ghana', 'Cameroon', 'DRC', 'Kenya', 'Madagascar', 'Uganda', 'Zambia', 'Mali', 'Congo', 'Tanzania', 'Djibouti']
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7.5))
for country in africa:
    df_country = get_country_data(df, country)
    df_country['Cases'].plot(label=country, ax=ax1)
    df_country['Deaths'].plot(label=country, ax=ax2)
ax1.set_xlim('03-01-2020', last_data_date)
ax1.set_title('Confirmed Cases')
ax1.grid(b=True, which='major', axis='both')
ax1.legend()
ax2.set_xlim('03-01-2020', last_data_date)
ax2.set_title('Deaths')
ax2.grid(b=True, which='major', axis='both')
ax2.legend()
ax1.set_xlabel(None)
ax2.set_xlabel(None)
ax1.tick_params(axis='both', which='both', labelsize=12)
ax2.tick_params(axis='both', which='both', labelsize=12)

## Has anyone learned from the earlier epidemics?
The hope is that countries which saw outbreaks later on in the pandemic may have been able to take strong measures to stop the outbreaks in their countries faster than we have seen in Western Europe and USA so far. This is hard to tell at this relatively early stage in the global epidemic but should become clear in a matter of weeks.

## The 'China Virus'

Hindsight is a powerful thing and it is particularly sobering to look back at the way the media reported the early days of the outbreak in China. As the cases grew almost exclusively in China for about a month, it was easy for us in the West to distance ourselves from a very real threat of an epidemic that would reach us some day - the numbers we saw were worrying but were still 'just in China'. The idea that Western nations could experience the same life-changing restrictions of quarantines and lock-downs or our societies to be crippled in such a devastating way seemed somewhat farsical. The media tells a rather different story today.

We are not near the end of this crisis but my hope is that a silver lining of this all will be to show people around the globe that events can change even the lives of the most privileged. Perhaps we will finally be awakened as one to respond to other potential global emergencies with the urgency that they too deserve in a similar way as we are seeing across the world today.